In [1]:
!pip install torchtext==0.8.0
!pip install transformers
!pip install nlp
!pip install sentencepiece
!pip install pytorch_lightning==0.9
!pip install rouge_score
!pip install install torch==1.7.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 13.2 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.2/408.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: pytorch_lightning
    Found existing installation: pytorch-lightning 1.6.5
    Uninstalling pytorch-lightning-1.6.5:
      Successfully uninstalled pytorch-lightning-1.6.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the follo

In [2]:
import torch
import random
import subprocess
import numpy as np
from tqdm.notebook import tqdm

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def install_dependencies(package_names: list) -> None:
    for i, package_name in enumerate(tqdm(package_names)):
        print(f"{i + 1}. pip install {package_name}", flush=True)
    subprocess.call(["pip", "install", package_name])

def print_items_from_dict(d:dict) -> None:
    print("Items held:")
    for item, amount in d.items():
        print(f"{item} ({amount})")

In [3]:
import logging
import pytorch_lightning as pl

logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
    def on_train_end(self, trainer, pl_module):
        logger.info("***** Train results *****")
        if pl_module.is_logger():
            metrics = trainer.callback_metrics
            # Log results
            for key in sorted(metrics):
                if key not in ["log", "progress_bar"]:
                    logger.info("{} = {}\n".format(key, str(metrics[key])))
    
    def on_validation_end(self, trainer, pl_module):
        logger.info("***** Validation results *****")
        if pl_module.is_logger():
            metrics = trainer.callback_metrics
            # Log results
            for key in sorted(metrics):
                if key not in ["log", "progress_bar"]:
                    logger.info("{} = {}\n".format(key, str(metrics[key])))

    def on_test_end(self, trainer, pl_module):
        logger.info("***** Test results *****")

        if pl_module.is_logger():
            metrics = trainer.callback_metrics

            # Log and save results to file
            output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
            with open(output_test_results_file, "w") as writer:
                for key in sorted(metrics):
                    if key not in ["log", "progress_bar"]:
                        logger.info("{} = {}\n".format(key, str(metrics[key])))
                        writer.write("{} = {}\n".format(key, str(metrics[key])))

In [4]:
import pandas as pd
from nlp import load_dataset
from transformers import T5Tokenizer
from torch.utils.data import Dataset

class mydataset(Dataset):
    def __init__(self, type_path: str, 
                       input_length: int, 
                       output_length: int,
                       num_samples: int = None,
                       tokenizer = T5Tokenizer.from_pretrained('t5-base')) -> None:      

        
        ################################################################
        self.dataset=pd.read_csv("/kaggle/input/smartworkt5/TrainingFeed_DW_06.08.csv",encoding= 'unicode_escape')

        ################################################################
        
        if num_samples:
            self.dataset = self.dataset.iloc[:num_samples]
        self.input_length = input_length
        self.tokenizer = tokenizer
        self.output_length = output_length
  
    def __len__(self) -> int:
        return self.dataset.shape[0]

    def clean_text(self, text: str) -> str:
        text = str(text)
        return text.replace('\n','').replace('``', '').replace('"', '')
    
    def convert_to_features(self, example_batch):   
        input_ = self.clean_text(example_batch['question'])
        target_ = self.clean_text(example_batch['query'])
        
        source = self.tokenizer.batch_encode_plus([input_], max_length=self.input_length, 
                                                     padding='max_length', truncation=True, return_tensors="pt")
        
        targets = self.tokenizer.batch_encode_plus([target_], max_length=self.output_length, 
                                                     padding='max_length', truncation=True, return_tensors="pt")   
        return source, targets
  
    def __getitem__(self, index: int) -> dict:
        source, targets = self.convert_to_features(self.dataset.iloc[index])
        
        source_ids = source["input_ids"].squeeze()
        target_ids = targets["input_ids"].squeeze()

        src_mask    = source["attention_mask"].squeeze()
        target_mask = targets["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}


def get_dataset(tokenizer, type_path: str, num_samples: int, args):
      return mydataset(type_path=type_path, 
                    num_samples=num_samples,  
                    input_length=args.max_input_length, 
                    output_length=args.max_output_length)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [5]:
import time
import torch
import numpy as np
from nlp import load_metric
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from typing import Callable, Dict, Iterable, List, Tuple, Union
#from dataset import get_dataset

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams) -> None:
        super(T5FineTuner, self).__init__()
        self.hparams = hparams        
        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name)
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name)
        self.rouge_metric = load_metric('rouge')

        
        if self.hparams.freeze_embeds:
            self.freeze_embeds()
        if self.hparams.freeze_encoder:
            self.freeze_params(self.model.get_encoder())
            self.assert_all_frozen(self.model.get_encoder())
            
            
        n_observations_per_split = {
            "train": self.hparams.n_train,
            "validation": self.hparams.n_val,
            "test": self.hparams.n_test,
        }
        self.n_obs = {k: v if v >= 0 else None for k, v in n_observations_per_split.items()}
        
    
    def freeze_params(self, model):
        for par in model.parameters():
            par.requires_grad = False
            
            
    def freeze_embeds(self):
        """Freeze token embeddings and positional embeddings for bart, just token embeddings for t5."""
        try:
            self.freeze_params(self.model.model.shared)
            for d in [self.model.model.encoder, self.model.model.decoder]:
                self.freeze_params(d.embed_positions)
                self.freeze_params(d.embed_tokens)
        except AttributeError:
            self.freeze_params(self.model.shared)
            for d in [self.model.encoder, self.model.decoder]:
                self.freeze_params(d.embed_tokens)
    
    def lmap(self, f: Callable, x: Iterable) -> List:
        """list(map(f, x))"""
        return list(map(f, x))

    def assert_all_frozen(self, model):
        model_grads: List[bool] = list(self.grad_status(model))
        n_require_grad = sum(self.lmap(int, model_grads))
        npars = len(model_grads)
        assert not any(model_grads), f"{n_require_grad/npars:.1%} of {npars} weights require grad"

    def is_logger(self):
        return self.trainer.global_rank  <= 0
    
    
    def parse_score(self, result):
        return {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}
        
    def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
  ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,
    )

    def _step(self, batch):
        labels = batch["target_ids"]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            labels=labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs[0]

        return loss
    
    
    def ids_to_clean_text(self, generated_ids):
        gen_text = self.tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
        return self.lmap(str.strip, gen_text)
    
    
    def _generative_step(self, batch) :
        
        t0 = time.time()
        
        generated_ids = self.model.generate(
            batch["source_ids"],
            attention_mask=batch["source_mask"],
            use_cache=True,
            decoder_attention_mask=batch['target_mask'],
            max_length=300, 
            num_beams=2,
            repetition_penalty=2.0, 
            length_penalty=0.5, 
            early_stopping=True
        )
        preds = self.ids_to_clean_text(generated_ids)
        target = self.ids_to_clean_text(batch["target_ids"])
            
        gen_time = (time.time() - t0) / batch["source_ids"].shape[0]  
    
        loss = self._step(batch)
        base_metrics = {'val_loss': loss}
        summ_len = np.mean(self.lmap(len, generated_ids))
        base_metrics.update(gen_time=gen_time, gen_len=summ_len, preds=preds, target=target)
        self.rouge_metric.add_batch(preds, target)
        
        return base_metrics
    

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss, "log": tensorboard_logs}
  
    def training_epoch_end(self, outputs):
        print("\n")
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}
        return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        
        return self._generative_step(batch)
    
  
    def validation_epoch_end(self, outputs):

        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        
        rouge_results = self.rouge_metric.compute() 
        rouge_dict = self.parse_score(rouge_results)
    
        tensorboard_logs.update(rouge1=rouge_dict['rouge1'], rougeL=rouge_dict['rougeL'])
        
        self.target_gen= []
        self.prediction_gen=[]
        return {"avg_val_loss": avg_loss, 
                "rouge1" : rouge_results['rouge1'],
                "rougeL" : rouge_results['rougeL'],
                "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def configure_optimizers(self):

        "Prepare optimizer and schedule (linear warmup and decay)"
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
        self.opt = optimizer
        return [optimizer]

    def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, second_order_closure=None, using_native_amp=None): 

        if self.trainer.use_tpu:
            xm.optimizer_step(optimizer)
        else:
          optimizer.step()
        optimizer.zero_grad()
        self.lr_scheduler.step()
  
    def get_tqdm_dict(self):

        tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}
        return tqdm_dict
    

    def train_dataloader(self):   
        n_samples = self.n_obs['train']
        train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", num_samples=n_samples, args=self.hparams)
        dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)
        t_total = (
            (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
            // self.hparams.gradient_accumulation_steps
            * float(self.hparams.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader

    def val_dataloader(self):
        n_samples = self.n_obs['validation']
        validation_dataset = get_dataset(tokenizer=self.tokenizer, type_path="validation", num_samples=n_samples, args=self.hparams)
        
        return DataLoader(validation_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)
    
    
    def test_dataloader(self):
        n_samples = self.n_obs['test']
        test_dataset = get_dataset(tokenizer=self.tokenizer, type_path="test", num_samples=n_samples, args=self.hparams)
        
        return DataLoader(test_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [6]:
set_seed(42)

In [7]:
import json
import argparse
from nlp import load_metric
#from model import T5FineTuner
import pytorch_lightning as pl
from transformers import T5Tokenizer
#from dataset import wikisql, get_dataset
#from callback import LoggingCallback, logger

In [8]:
dataset = mydataset(type_path='validation',
                  input_length=512,
                  output_length=150)
len(dataset)

10360

In [9]:
cfg = {
"output_dir":"t5_wikisql",
"model_name":"t5-base",
"tokenizer_name":"t5-base",
"max_input_length":512,
"max_output_length":150,# 300: kaggle can't handle more than 150 
"freeze_encoder":False,
"freeze_embeds":False,
"learning_rate":0.003,
"weight_decay":0,
"adam_epsilon":1e-8,
"warmup_steps":0,
"train_batch_size":8, # 2 total : 10,000 queries , 80%: training = 80k /8 train_batch_size = 1,000 times --> model runs 1,000 times 
"eval_batch_size":4, # 2 (2,000 queries / 4 eval_batch_size = 500 times --> model runs 1,000 times) 
"num_train_epochs":8, # kaggle allows 10 hours =  apx 25 epochs
"gradient_accumulation_steps":1,
"n_gpu":1,
"resume_from_checkpoint":None,
"val_check_interval":0.8,
"n_val":1000,
"n_train":-1,
"n_test":-1,
"early_stop_callback":False,
"fp_16":False,
"opt_level":"O1",
"max_grad_norm":1,
"seed":42,
"automatic_optimization":False
}
with open('config.json', 'w') as f:
    json.dump(cfg, f)
    
    

In [10]:
with open('config.json') as f:
  args_dict = json.load(f)
args = argparse.Namespace(**args_dict)
print_items_from_dict(args_dict)

Items held:
output_dir (t5_wikisql)
model_name (t5-base)
tokenizer_name (t5-base)
max_input_length (512)
max_output_length (150)
freeze_encoder (False)
freeze_embeds (False)
learning_rate (0.003)
weight_decay (0)
adam_epsilon (1e-08)
warmup_steps (0)
train_batch_size (8)
eval_batch_size (4)
num_train_epochs (8)
gradient_accumulation_steps (1)
n_gpu (1)
resume_from_checkpoint (None)
val_check_interval (0.8)
n_val (1000)
n_train (-1)
n_test (-1)
early_stop_callback (False)
fp_16 (False)
opt_level (O1)
max_grad_norm (1)
seed (42)
automatic_optimization (False)


In [11]:
from pytorch_lightning.callbacks import ModelCheckpoint

In [12]:
## Define Checkpoint function
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filepath=args.output_dir, prefix="checkpoint", monitor="val_loss", mode="min", save_top_k=3
)

## If resuming from checkpoint, add an arg resume_from_checkpoint
train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    resume_from_checkpoint=args.resume_from_checkpoint,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    val_check_interval=args.val_check_interval,
    logger=None,
    callbacks=[LoggingCallback()],
)

In [13]:
model = T5FineTuner(args)

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Downloading:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

In [14]:
trainer = pl.Trainer(**train_params)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [15]:
trainer.fit(model)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Saving latest checkpoint..


1

In [16]:
model.model.save_pretrained('/kaggle/working/model_T5.bin')

In [17]:
!pip install Sentencepiece
!pip install transformers

In [18]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [19]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/opt/conda/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [20]:
import re
import pandas as pd
from difflib import SequenceMatcher
result=[]

df = pd.read_csv('/kaggle/input/smartworkt5/Testset2_904_06.08.csv', encoding = 'ISO-8859-1')
for i in range(len(df)): 
    query = df.iloc[i,5] #query is "Question" (Natural Language texts)
    features = tokenizer.batch_encode_plus([query],  return_tensors="pt")
    output = model.model.generate(input_ids=features['input_ids'], 
                        attention_mask=features['attention_mask'],max_length=300)
    output1=tokenizer.decode(output[0]).lstrip("<pad>")
    output2=output1.rstrip("</s>").strip()
    output3=output2.replace('\n', ' ')
    uppercased=output3.upper()
    RS_UC=uppercased.replace(' ', '')
     
    print(i)
    #######################
    df.iloc[i,2] = df.iloc[i,2].replace('\n', ' ')
    UC_Gen = df.iloc[i,2].upper()
    RS_UC_Gen = UC_Gen.replace(' ', '')

    ratio1 = SequenceMatcher(None, re.sub(' +', ' ', RS_UC_Gen), RS_UC).ratio() ##
    result.append((df.iloc[i,0], df.iloc[i,5], df.iloc[i,2].strip(), output3, ratio1)) ##

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [21]:
pd.set_option("display.max_colwidth", 100)

pd.options.display.max_columns = 4
pd.set_option("display.min_rows", 100)
A=pd.DataFrame([x[0] for x in result],columns=['Query Number'])
B=pd.DataFrame([x[1] for x in result],columns=['Question'])
C=pd.DataFrame([x[2] for x in result],columns=['Target SQL'])
D=pd.DataFrame([x[3] for x in result],columns=['Generated SQL'])
E=pd.DataFrame([x[4] for x in result],columns=['Accuracy'])

df = pd.concat([A,B,C,D,E],axis=1)
df

,Query Number,Question,...,Generated SQL,Accuracy
0,1239.0,How many location numbers are there in each location class?,...,"SELECT LOCATION_CLASS, COUNT(LOCATION_NO) COUNT(LOCATION_NO) FROM DW_LOCATION GROUP BY LOCATION_...",0.892857
1,1239.1,Show me the count of location no by location class,...,SELECT COUNT(LOCATION_CLASS)) FROM DW_LOCATION GROUP BY LOCATION_CLASS,0.834532
2,1240.0,Show me the number of locations in Winnipeg.,...,SELECT COUNT(DISTINCT(LOCATION_NO)) FROM DW_LOCATION WHERE CITY = 'WINNIPEG',1.000000
3,1240.1,What's the count of location in Winnipeg,...,SELECT COUNT(DISTINCT(LOCATION_NO)) FROM DW_LOCATION WHERE CITY = 'WINNIPEG',1.000000
4,1241.0,What is the number of location classes found?,...,SELECT count(DISTINCT(LOCATION_CLASS)) FROM DW_LOCATION,1.000000
5,1242.0,Count number of location class that exists in MB.,...,SELECT COUNT(DISTINCT(LOCATION_CLASS)) FROM DW_LOCATION WHERE STATE = 'MB',1.000000
6,1242.1,Show me the count of location class in MB.,...,SELECT COUNT(DISTINCT(LOCATION_CLASS)) FROM DW_LOCATION WHERE CITY = 'MB',0.947368
7,1243.0,How many units are in 804 Willington Terrace?,...,SELECT COUNT(DISTINCT(UNIT_NO)) FROM DW_LOCATION WHERE STREET_NO = '804' AND STREET_NAME = 'WILL...,1.000000
8,1244.0,Is there any street number that has more than 100 units?,...,"SELECT DISTINCT 'Y' FROM DW_LOCATION L2 WHERE EXISTS (SELECT STREET_NO, COUNT(DISTINCT(UNIT_NO))...",1.000000
9,1245.0,What is the maximum number of units in one street no.?,...,"SELECT MAX(COUNT_UNIT_NO) FROM (SELECT POSTAL_CD, STREET_NO, STREET_NO, STREET_NO, COUNT(UNIT_NO...",0.884615


In [22]:
df.to_csv('model_output_06.08_test1_Lien.csv', index=False)

In [23]:
df['Accuracy'].mean()

0.9947439968305741

In [24]:
df = pd.read_csv('model_output_06.08_test1_Lien.csv')